In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc4 as pm
import tensorflow as tf
import arviz as az

### Linear Regression with General Covariance-Matrix

In [ ]:
N = 2000
k = 3
beta = np.array([[0.5], [2.5], [-3.]])
X = 10 * np.random.randn(N, k)
y = np.dot(X, beta) + np.random.randn(N, 1)


In [ ]:
fig, axes = plt.subplots(1,3, figsize=(16,4))
axes[0].plot(y, X[:,0], '+')
axes[1].plot(y, X[:,1], '+')
axes[2].plot(y, X[:,2], '+')

In [ ]:
help(pm.sample)

In [ ]:
@pm.model
def linear_model():
    alpha = yield pm.Normal(loc=0, scale=10, name='alpha', batch_stack=1)
    beta0 = yield pm.Normal(loc=0, scale=10, name='beta0', batch_stack=1)
    beta1 = yield pm.Normal(loc=0, scale=10, name='beta1', batch_stack=1)
    beta2 = yield pm.Normal(loc=0, scale=10, name='beta2', batch_stack=1)
    sigma = yield pm.HalfNormal(scale=1, name='sigma', batch_stack=1)
    mu = alpha + beta0 * X[:,0] + beta1 * X[:,1] + beta2 * X[:,2]
    
    y_obs = yield pm.Normal(loc=mu, scale=sigma, observed=y.ravel(), name='y_obs')
    
    return y_obs
    
trace = pm.sample(linear_model(), num_samples=10000, num_chains=1, burn_in=1000)

In [ ]:
az.plot_trace(trace);

In [ ]:
N = 200

# we'll use a dynamics matrix with a decaying rotational component plus a small perturbation
theta = 5. / 180 * np.pi
rot_comp = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
true_A = 0.9 * rot_comp + 0.05 * np.random.randn(2, 2)
true_sigma = 0.05
true_tau = 1./true_sigma**2
true_x0 = np.zeros(2)

x = np.zeros((N, 2))
x[0] = true_x0

for t in range(1, N):
    x[t] = np.dot(true_A, x[t-1].T).T + np.random.randn(2) * true_sigma


plt.plot(x)

In [ ]:
G = np.array([[1, 0, 1, 0, 0]])
F = np.array([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, -1, -1, -1], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]])

# V
psiy = 3
V = np.array([[1/psiy]])
chol_V = np.linalg.cholesky(V)

# W
psi1 = 7
psi2 = 5e5
psi3 = 5e5
psi4 = 1e16
psi5 = 1e16
W = np.eye(5)
np.fill_diagonal(W, np.array([1/psi1, 1/psi2, 1/psi3, 0, 0]))

#chol_W = np.linalg.cholesky(W)
chol_W = np.eye(5)
np.fill_diagonal(chol_W, np.array([1/psi1, 1/psi2, 1/psi3, 0, 0]))


# Sample Size
T = 150
# inital state
x0 = np.array([0, 0.2, 0, 0, 0])
x0[2:5] = 3*np.sin(np.linspace(-1, 1, 4))[0:3]
x = np.zeros((T, 5))
x[0] = x0
y = np.zeros((T, 1))

for t in range(1, T):
    x[t] = np.dot(F, x[t-1].T).T + np.dot(chol_W,  np.random.randn(chol_W.shape[0]))
    y[t] = np.dot(G, x[t].T).T + np.dot(chol_V,  np.random.randn(chol_V.shape[0]))

plt.plot(y)

\begin{align}
    y_{t} &= G_{t}x_{t} + X_{t}\beta_{t} + \nu_{t} \\
    x_{t} &= F_{t}x_{t-1} + Z_{t}\gamma_{t} + \omega_{t} \\
    \nu_{t} &\sim \mathcal{N}(0, V_{t}) \\
    \omega_{t} &\sim \mathcal{N}(0, W_{t}) \\
\end{align}

In [ ]:
from pymc3 import MvNormal, Normal, Flat, Continuous
from scipy.linalg import solve_triangular
from scipy.linalg import block_diag
import pdb
class StateSpaceModel(Continuous):
    """
    A state space model with Gaussian noise.
    
    This models only the state variables so that the form of the observation
    noise can be specified separately.
    
    Parameters
    ----------
    G : tensor
        tau > 0, innovation precision
    X : tensor
        sd > 0, innovation standard deviation (alternative to specifying tau)
    beta : tensor
        state update matrix
    V : 
    
    F : tensor
        input matrix
    Z : tensor
        (time x dim), inputs to the system
        init : distribution
        distribution for initial value (defaults to Flat())
    gamma :   
    W :
    
    State Space Equation
    ----------
    y_t = X_t * beta_t + G_t * x_t + vega_t  vega_t ~ N(0, V_t)
    eta_t = Z_t * gamma_t + F_t * eta_t-1 + omega_t  omega_t ~ N(0, W_t)
    """
    def __init__(self, 
                 G=None, X=None, beta=None, V=None, 
                 F=None, Z=None, gamma=None, W=None,
                 y = None, 
                 TT=1, init=Flat.dist(), *args, **kwargs):
        super(StateSpaceModel, self).__init__(*args, **kwargs)
        
        self.G = G
        self.X = X
        self.beta = beta
        self.V = V
        
        self.F = F
        self.Z = Z
        self.gamma = gamma
        self.W = W
        
        self.Y = y
        #self.mean = 0.
        #self.x0 = x0
        
        self.q = F.shape[0]
        self.T = TT
        self.Tq = self.T*self.q
        
        self.Omega11inv = np.linalg.inv(V)
        
        # S
        self.DDinv = np.eye(self.q)
        self.Omega22inv = np.eye(self.q)
        self.Sinv = block_diag(self.DDinv, np.kron(np.eye(self.T-1), self.Omega22inv))
        
        # H
        H1 = np.eye(self.Tq)
        H2 = np.vstack((np.zeros((self.q, self.Tq)), np.hstack((np.kron(np.eye(self.T-1), F), np.zeros((self.Tq-self.q,self.q))))))
        self.bigH = H1 - H2
        self.bigHt = self.bigH.T   
        
        # G
        self.bigG = np.kron(np.eye(self.T), self.G)
        self.bigGt = self.bigG.T
        #pdb.set_trace()
        self.K = self.bigHt.dot(self.Sinv).dot(self.bigH)

        self.GtOmega11 = self.bigGt.dot(np.kron(np.eye(self.T), self.Omega11inv))
        self.GtOmega11G = self.GtOmega11.dot(self.bigG)
        self.GtOmega11Y = self.GtOmega11.dot(self.Y)

        P = self.K + self.GtOmega11G;
        self.P = (P+P.T)/2
        self.L = np.linalg.cholesky(self.P)

        self.eta_hat = solve_triangular(self.L.T, solve_triangular(self.L, self.GtOmega11Y, lower=True), lower=False)
        
            
    def random(self, point=None, size=None):
        tau, sd, A, B, u, init = draw_values([self.tau, self.sd, self.A, self.B, self.u, self.init], point=point)
        
        T, D = size
        x = np.zeros(T, D)
        x[0,:] = init
        
        for t in range(1, T):
            x[t,:] = np.dot(A, x[t-1,:].T).T + np.dot(B, u[t-1,:].T).T + np.random.randn(1, D) * sd
            
        return x
    
    def logp(self, x):
        G = self.G
        V = self.V
        F = self.F
        W = self.W

        eta_hat = self.eta_hat
        P = self.P
        L = self.L
        x = np.zeros((self.Tq), np.float64)
        
        innov_like = MvNormal.dist(mu=eta_hat.reshape(self.Tq), chol=L, lower=True).logp(theano.shared(x))
        return innov_like

In [ ]:
MvNormal.dist(mu=np.zeros(750), chol=np.eye(750), lower=True).logp(theano.shared(x.reshape(750))).eval()

In [ ]:
with mc.Model() as model:
    psiy = mc.Gamma('psiy', alpha=1e-3, beta=1e-3)
    V = np.array([[1/psiy]])
    X = StateSpaceModel('X', F=F, G=G, V=V, W=W, y=y, TT=150, observed=x.reshape(750))
    
    trace = mc.sample(1000)

In [ ]:
G = np.array([[1, 0, 1, 0, 0]])
F = np.array([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, -1, -1, -1], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]])

# V
psiy = 3
Omega11 = np.array([[1/psiy]])
chol_Omega11 = np.linalg.cholesky(Omega11)

# W
psi1 = 7
psi2 = 5e5
psi3 = 5e5
psi4 = 1e16
psi5 = 1e16
Omega22 = np.eye(5)
np.fill_diagonal(Omega22, np.array([1/psi1, 1/psi2, 1/psi3, 0, 0]))

#chol_W = np.linalg.cholesky(W)
chol_Omega22 = np.eye(5)
np.fill_diagonal(chol_Omega22, np.array([1/np.sqrt(psi1), 1/np.sqrt(psi2), 1/np.sqrt(psi3), 0, 0]))
q=5
TT=150
TTq=TT*q

with mc.Model() as model:

        sd_dist = mc.HalfCauchy.dist(beta=2.5)
        packed_chol = mc.LKJCholeskyCov('chol_cov', eta=20, n=15, sd_dist=sd_dist)
        Omega22 = mc.expand_packed_triangular(15, packed_chol, lower=True)
        
        psiy = mc.InverseGamma('psiy', alpha=1e-3, beta=1e-3, shape=(1,1))

        # S
        DDinv = np.eye(q)
        Omega22inv = mc.math.matrix_inverse(Omega22)
        Sinv = block_diag(DDinv, np.kron(np.eye(TT-1), Omega22inv))
        
        # H
        H1 = np.eye(TTq)
        H2 = np.vstack((np.zeros((q, TTq)), np.hstack((np.kron(np.eye(TT-1), F), np.zeros((TTq-q, q))))))
        bigH = H1 - H2
        bigHt = bigH.T   
        
        # G
        bigG = mc.math.kronecker(np.eye(TT), G)
        bigGt = bigG.T
        K = mc.math.dot(bigHt, mc.math.dot(Sinv, bigH))

        GtOmega11 = mc.math.dot(bigGt, mc.math.kronecker(np.eye(TT), Omega11inv))
        GtOmega11G = mc.math.dot(GtOmega11, bigG)
        GtOmega11Y = mc.math.dot(GtOmega11, Y)

        P = K + GtOmega11G;
        P = (P+P.T)/2
        L = np.linalg.cholesky(P)
        
        eta_hat = solve_triangular(L.T, solve_triangular(L, GtOmega11Y, lower=True), lower=False)
        
        X = mc.MvNormal('X', mu=eta_hat, tau=P, shape = TTq)
    
        trace = mc.sample(10)
    


In [ ]:
with mc.Model() as model:
    # Note that we access the distribution for the standard
    # deviations, and do not create a new random variable.
    sd_dist = mc.HalfCauchy.dist(beta=2.5)
    packed_chol = mc.LKJCholeskyCov('chol_cov', eta=2, n=10, sd_dist=sd_dist)
    chol = mc.expand_packed_triangular(10, packed_chol, lower=True)

    # Define a new MvNormal with the given covariance
    vals = mc.MvNormal('vals', mu=np.zeros(10), chol=chol, shape=10)

    # Or transform an uncorrelated normal:
    vals_raw = mc.Normal('vals_raw', mu=0, sigma=1, shape=10)
    vals = T.dot(chol, vals_raw)

    # Or compute the covariance matrix
    cov = T.dot(chol, chol.T)

    # Extract the standard deviations
    stds = T.sqrt(T.diag(cov))
    trace = mc.sample(10)
    

In [ ]:
mc.traceplot(trace, compact=False)

In [ ]:
G = np.array([[1, 0, 1, 0, 0]])
F = np.array([[1, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, -1, -1, -1], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0]])

# V
psiy = 3
Omega11 = np.array([[1/psiy]])
chol_Omega11 = np.linalg.cholesky(Omega11)

# W
psi1 = 7
psi2 = 5e5
psi3 = 5e5
psi4 = 1e16
psi5 = 1e16
Omega22 = np.eye(5)
np.fill_diagonal(Omega22, np.array([1/psi1, 1/psi2, 1/psi3, 0, 0]))

#chol_W = np.linalg.cholesky(W)
chol_Omega22 = np.eye(5)
np.fill_diagonal(chol_Omega22, np.array([1/np.sqrt(psi1), 1/np.sqrt(psi2), 1/np.sqrt(psi3), 0, 0]))
q=5
TT=150
Tq=750

with mc.Model() as model:

        sd_dist = mc.HalfCauchy.dist(beta=2.5)
        packed_chol = mc.LKJCholeskyCov('chol_cov', eta=20, n=15, sd_dist=sd_dist)
        Omega22 = mc.expand_packed_triangular(15, packed_chol, lower=True)
               
        X = mc.MvNormal('X', mu=np.zeros(5), tau=Omega22)
    
        trace = mc.sample(10)
    



In [ ]:
np.fill_diagonal(cholOmega22, np.array([1/np.sqrt(psi1), 1/np.sqrt(psi2), 1/np.sqrt(psi3), 0, 0]))

In [ ]:
cov = np.array([[1., 0.5], [0.5, 2]])
mu = np.array([[1, 9, -1, -3]])

C = np.kron(np.eye(2), cov)

with mc.Model() as model:
    A = mc.MvNormal('A', mu=mu, cov=C, shape=(1,4))
    
    trace = mc.sample(1000)

In [ ]:
trace['A'].shape

In [ ]:
mc.traceplot(trace, compact=False)

In [ ]:
trace['A'][:,0,:]

In [ ]:
plt.scatter(trace['A'][:,0,0], trace['A'][:,0,1])
plt.show()

In [ ]:
true_sig_o = 0.2
true_tau_o = 1./true_sig_o**2
true_C = np.random.randn(1, 2)
y = np.random.randn(x.shape[0], true_C.shape[0]) * true_sig_o + np.dot(true_C, x.T).T

plt.plot(y)



In [ ]:
with mc.Model() as model:
    A = mc.Normal('A', mu=np.eye(2), tau=1e-5, shape=(2,2))
    Tau = mc.Gamma('tau', mu=100, sd=100)
    
    X = StateSpaceModel('x', A=A, B=T.zeros((1,1)), u=T.zeros((x.shape[0],1)), tau=Tau, shape=(y.shape[0], 2))
    
    C = mc.Normal('C', mu=np.zeros((1,2)), tau=1e-5, shape=(1,2))
    Tau_o = mc.Gamma('tau_o', mu=100, sd=100)
    
    Y = mc.Normal('y', mu=T.dot(C, X.T).T, tau=Tau_o, observed=y)
    
    trace = mc.sample(10000)
    #inference = mc.ADVI()
    #approx = mc.fit(n=100000, method=inference)

In [ ]:
mc.traceplot(trace, compact=False)

In [ ]:
mc.plot_posterior(approx.sample(10000), color='LightSeaGreen');